***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-1b01  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from operator import add

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  5 16:06 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
%cd -q /home/dataproc
!ls inverted_index_text_gcp.py

inverted_index_text_gcp.py


In [8]:
from inverted_index_text_gcp import *

InvertedIndex.DIR_NAME = "text_index/postings_gcp_text_index"

In [9]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket-ir-project-nicoleayelet' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name.startswith('multi'):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [10]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [11]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_text_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
inverted_text = InvertedIndex()
inverted_text.DIR_NAME = InvertedIndex.DIR_NAME

In [14]:
# Count number of wiki pages
N_docs = parquetFile.count()

inverted_text.N = N_docs

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [15]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

DIR_NAME = "text_index/postings_gcp_text_index"

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE

  # count frequency of each word that is not in stopwords
  word_count_dict = {}
  for token in tokens:
      if token not in all_stopwords:
          if token not in word_count_dict:
              word_count_dict[token] = 1
          else:
              word_count_dict[token] += 1

  result = [(word, (id, count)) for word, count in word_count_dict.items()] # create a list of tuples in form (token, (doc_id, tf))

  return result


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE

  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])

  return sorted_pl


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE

  flattened_postings = postings.flatMap(lambda x: set((x[0], doc_id) for doc_id, _ in x[1]))

  # Count the occurrences of each token (document frequency)
  token_df = flattened_postings.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)

  return token_df


def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # YOUR CODE HERE

  buckets = postings.map(lambda x: (token2bucket_id(x[0]), x))

  # Group posting lists in each bucket
  grouped_buckets = buckets.groupByKey()

  # Write posting lists to disk and collect information about the locations
  def write_and_get_info(bucket):
      bucket_id, posting_list_iter = bucket
      postings_info = InvertedIndex.write_a_posting_list((bucket_id, posting_list_iter), bucket_name)
      return postings_info

  posting_locations = grouped_buckets.map(write_and_get_info)

  return posting_locations


def count_doc_len(doc_id, text):
    '''this functen calculates the document len given the dic id and text '''

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    num_tokens = len([x for x in tokens if x not in all_stopwords])
    return (doc_id,num_tokens)

In [16]:
# Perform word count on document-text pairs and aggregate by word.
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# Filter out words with total counts less than or equal to 50.
postings_filtered = postings.filter(lambda x: len(x[1])>50)


In [17]:
# Calculate total frequency for each term and store in 'term_total'.
total_terms_text = postings_filtered.flatMapValues(lambda x: x).map(lambda x: (x[0], x[1][1])).reduceByKey(add)
inverted_text.term_total = total_terms_text.collectAsMap()


In [18]:
# Calculate and store document lengths: {doc_id: doc_length}.
len_docs_text = doc_text_pairs.map(lambda x: count_doc_len(x[1], x[0]))
len_docs_text = len_docs_text.collectAsMap()
inverted_text.DL = len_docs_text


In [19]:
# filtering postings and calculate df for each term. stores it in text_index.df attribute
w2df_text = calculate_df(postings_filtered)
w2df_text_dict = w2df_text.collectAsMap()

In [20]:
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()


In [22]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=DIR_NAME):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

Putting it all together

In [23]:
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs

# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_text_dict

# write the global stats out
inverted_text.write_index('.', 'text_index')

# upload to gs
index_src = "text_index.pkl"
index_dst = f'gs://{bucket_name}/text_index/postings_text_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://text_index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 69.0 MiB/ 69.0 MiB]                                                
Operation completed over 1 objects/69.0 MiB.                                     
